In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%pip install ultralytics
%pip install supervision

from ultralytics import YOLO
from pathlib import Path
import supervision as sv
import cv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
# Model ve kaynak ayarları
byte_tracker = sv.ByteTrack()
annotator = sv.BoxAnnotator()
MODEL_PATH = "/content/gdrive/MyDrive/ground-garbage-yoluv8m-i256-e100.pt"
model = YOLO(MODEL_PATH)
SOURCE = Path("/content/gdrive/MyDrive/Colab Notebooks/test-data")

output_dir = Path("/content/gdrive/MyDrive/output4")
output_dir.mkdir(parents=True, exist_ok=True)

# Sonuçları stream olarak alıyoruz
results = model(SOURCE, stream=True, imgsz=256, conf=0.4, max_det=1, task="detect")

frame_counter = 0
total_containers = 0

for result in results:
    frame_counter += 1
    detections = sv.Detections.from_ultralytics(result)

    container_count = len(detections.xyxy)
    total_containers += container_count

    # Güven skoru yüzdesi ve bboxları alıyoruz
    confidences = detections.confidence * 100

    # Görüntünün üzerine bbox ve güven skoru yazdırma
    annotated_frame = annotator.annotate(
        scene=result.orig_img,
        detections=detections
    )

    # Her bbox üzerine doğruluk oranını yazdır
    for i, bbox in enumerate(detections.xyxy):
        confidence_label = f"{confidences[i]:.2f}%"
        cv2.putText(
            annotated_frame,
            confidence_label,
            (int(bbox[0]), int(bbox[1]) - 10),  # Bbox'ın üstüne yerleştirmek için
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2
        )

    # Güncellenmiş görüntüyü kaydediyoruz
    output_path = f"/content/gdrive/MyDrive/output4/frame_{frame_counter}.jpg"
    cv2.imwrite(output_path, annotated_frame)
    print(f"Frame {frame_counter}: {container_count} containers detected")
    for i, confidence in enumerate(confidences):
        print(f"Frame {frame_counter}, Detection {i+1}: Confidence {confidence:.2f}%")

print(f"\nTotal containers detected in all images: {total_containers}")


image 1/55 /content/gdrive/MyDrive/Colab Notebooks/test-data/34KRH236 - CH2-2024-9-4-23-42-22.png: 224x256 (no detections), 250.8ms
Frame 1: 0 containers detected
image 2/55 /content/gdrive/MyDrive/Colab Notebooks/test-data/34KRH352 - CH2-2024-9-4-23-10-35.png: 224x256 1 garbage, 253.9ms
Frame 2: 1 containers detected
Frame 2, Detection 1: Confidence 63.24%
image 3/55 /content/gdrive/MyDrive/Colab Notebooks/test-data/34KRH352 - CH2-2024-9-4-23-13-47.png: 224x256 1 garbage, 249.0ms
Frame 3: 1 containers detected
Frame 3, Detection 1: Confidence 80.57%
image 4/55 /content/gdrive/MyDrive/Colab Notebooks/test-data/34KRH352 - CH2-2024-9-4-23-14-15.png: 224x256 1 garbage, 244.7ms
Frame 4: 1 containers detected
Frame 4, Detection 1: Confidence 81.82%
image 5/55 /content/gdrive/MyDrive/Colab Notebooks/test-data/34KRH352 - CH2-2024-9-4-23-14-28.png: 224x256 1 garbage, 244.2ms
Frame 5: 1 containers detected
Frame 5, Detection 1: Confidence 74.25%
image 6/55 /content/gdrive/MyDrive/Colab Noteboo

In [ ]:
import shutil
shutil.make_archive("/content/gdrive/MyDrive/output_files", 'zip', "/content/gdrive/MyDrive/output4")

from google.colab import files
files.download("/content/gdrive/MyDrive/output_files.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>